##10_교차검증&앙상블: 부스팅

###1. 데이터셋 불러오기

In [1]:
import pandas as pd
#https://www.kaggle.com/kartikmohan1999/universal-bank/data?select=UniversalBank.csv
bank_df = pd.read_csv('UniversalBank.csv')
bank_df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


학습에 사용할 특성변수 선택하기

In [2]:
X = bank_df.drop(['ID', 'ZIP Code', 'Personal Loan'], axis = 1)
y = bank_df['Personal Loan']

###3. 데이터 분할

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1, stratify = y)

###5. 모델 추정: 부스팅 & 교차검증

####1. 학습에 사용할 모델 정의: 결정 트리

In [4]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 1, criterion = 'entropy', random_state = 1)

####2. 배깅 앙상블: 결정 트리

아다부스트(Ada Boost)

In [5]:
from sklearn.ensemble import AdaBoostClassifier

# n_estimators=500 : 트리가 500개
adaboost = AdaBoostClassifier(base_estimator = tree, n_estimators = 500, learning_rate = 0.1, random_state = 1)

####3. K-fold 교차 검증

결정 트리와 아다부스트 비교

In [6]:
from sklearn.model_selection import cross_val_score

clf_labels = ['Decision Tree', 'Ada Boost']
all_clf = [tree, adaboost]

for clf, label in zip(all_clf, clf_labels):
  scores = cross_val_score(estimator = clf, X = X_train, y = y_train, cv = 10, scoring = 'roc_auc') # cv=10 : k-fold가 10개
  print("ROC AUC: %0.3f (+/- %0.3f) [%s]", (scores.mean(), scores.std(), label))

ROC AUC: %0.3f (+/- %0.3f) [%s] (0.8829135713666967, 0.023406169666276122, 'Decision Tree')
ROC AUC: %0.3f (+/- %0.3f) [%s] (0.9835566978095359, 0.010774714837632118, 'Ada Boost')
